<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/develop/test_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Test Environment
Use this notebook to conduct single tests on the model of your choice.





---



## Setup
To import the model into the test environment enter the parameters into the script below. Use the following table to find the right entries for each model:


|Model|Context|MODEL_NAME|DATABASE|PARAMS|Note|
|----------|----------|----------|----------|----------|----------|
|Tiny Llama|-|tiny_llama_no_retriever|-|-|**very slow**|
|Tiny Llama|Whole documents|tiny_llama_dense|**external**|-|**very slow**|
|Mistral Instruct|-|mistral_instruct_no_retriever|-|-|**Huggingface and Pinecone Token and GPU required**|
|Mistral Instruct|Whole documents|mistral_instruct_dense|**external**|-|**Huggingface and Pinecone Token and GPU required**|
|Mistral Instruct|Text fragments|mistral_instruct_hybrid|sparse-dense|k, c, o|**Huggingface and Pinecone Token and GPU required**|
|Bert Base|QA Pairs|bert_base_qa_embeddings|**directly imported**|k|-|
|Bert Base|Whole documents|bert_base_dense|**external**|-|-|
|Bert Base|Text fragments|bert_base_sparse|sparse|k|-|
|Bert Finetuned|Whole documents|bert_finetuned_dense|**external**|-|-|
|Chat GPT 4o|-|chat_gpt_no_retriever|-|t|**Not free to use, OpenAI Token required**|
|Chat GPT 4o|Text fragments|chat_gpt_hybrid|sparse-dense|k, c, o, t|**Not free to use, OpenAI Token required**|

The meaning of each parameter can be found in this table

|Parameter Name|Description|
|----------|----------|
|k|Number of contexts the retriever sends to the reader|
|c|chunk size of each context|
|o|overlap between the contexts|
|t|temparature of the reader model|

If the model uses an external database, a directly imported database or no database please enter an empty string ("") for the DATABASE variable in the script.

In [1]:
import os
import importlib

os.environ["MODEL_NAME"] = "bert_base_sparse"
os.environ["DATABASE"] = "sparse"

Execute the following script to setup the test environment

In [2]:
import os
import shutil

# Set environment variables
directory = os.environ["MODEL_NAME"]
database = os.environ["DATABASE"]

# Install Git LFS
!sudo apt-get install git-lfs -y
!git lfs install

# Clone the repository
!git clone --branch develop https://github.com/dimitrod/ehu_nlp_dimathina.git
%cd ehu_nlp_dimathina

# Fetch and checkout files for the model
dir_path = f"models/{directory}/*"
!git lfs fetch --include="{dir_path}"
!git lfs checkout
%cd ..

# Move the model to the current directory
shutil.move(f"ehu_nlp_dimathina/models/{directory}", ".")

# Install model-specific requirements
!pip install -r {directory}/requirements.txt

# Handle the optional database
if database:
    %cd ehu_nlp_dimathina
    db_path = f"databases/{database}/*"
    !git lfs fetch --include="{db_path}"
    !git lfs checkout
    %cd ..
    shutil.move(f"ehu_nlp_dimathina/databases/{database}", f"{directory}/database")

# Cleanup
shutil.rmtree("ehu_nlp_dimathina")
shutil.rmtree("sample_data")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Git LFS initialized.
Cloning into 'ehu_nlp_dimathina'...
remote: Enumerating objects: 1813, done.
remote: Counting objects: 100% (551/551), done.
remote: Compressing objects: 100% (448/448), done.
remote: Total 1813 (delta 295), reused 146 (delta 103), pack-reused 1262 (from 1)
Receiving objects: 100% (1813/1813), 29.66 MiB | 21.69 MiB/s, done.
Resolving deltas: 100% (1108/1108), done.
Filtering content: 100% (28/28), 1.43 GiB | 43.60 MiB/s, done.
/content/ehu_nlp_dimathina
fetch: Fetching reference refs/heads/develop
Checking out LFS objects: 100% (38/38), 1.5 GB | 0 B/s, done.
/content
/content/ehu_nlp_dimathina
fetch: Fetching reference refs/heads/develop
Checking out LFS objects: 100% (38/38), 1.5 GB | 0 B/s, done.
/content


Enter the hyperparameters of the model. If the model doesn't require any leave the list empty **but still execute the script**

In [3]:
params = [5]

Create an instance of the model

In [4]:
model_name = os.environ["MODEL_NAME"]

module_path = f"{model_name}.{model_name}"  # Combine directory and module
module_obj = importlib.import_module(module_path)  # Import the module dynamically
cls_obj = getattr(module_obj, model_name)
model = cls_obj(params)

loading vecorizer
loading vector base
loading documents


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


loading model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu




---



# Tests
Start the tests that you want to conduct

In [5]:
question = "What is the capital of france?"

answer = model.invoke(question)

print(answer)

Brooklyn
